In [192]:
import argparse
import os
import sys
import timeit
import glob
import pandas as pd
import numpy as np
from rdkit import Chem

### 비교하고자 하는 레퍼런스 물질들

In [211]:
# csv file column : smiles
# write / at the end of path

path = '/Users/siu'
os.chdir(path)

In [214]:
csv = 'MP303_GOSTAR_actives.csv'
smiles = 'smiles'
names = 'compound_name'

In [215]:
from rdkit import DataStructs

df = pd.read_csv(csv)
names = df[names]
smiles = df[smiles]
mols = [Chem.MolFromSmiles(x) for x in smiles]
fps = [Chem.RDKFingerprint(x) for x in mols]
print(len(smiles), len(mols), len(fps))

304 304 304


In [216]:
ref = pd.DataFrame()
ref['smiles'] = smiles
ref['name'] = names
ref['mol'] = mols
ref['fp'] = fps

### 체크하고자 하는 물질들 (csv로 출력될)

In [217]:
csv = 'MP303_PD_3DShape_Top_PM_Screen.csv'
smiles = 'smiles'
names = 'compound_id'

In [218]:
from rdkit import DataStructs

df = pd.read_csv(csv)
mols = [Chem.MolFromSmiles(x) for x in df[smiles]]
fps = [Chem.RDKFingerprint(x) for x in mols]
print(len(df[smiles]), len(mols), len(fps))

297 297 297


In [219]:
scr = pd.DataFrame()
scr['smiles'] = smiles
scr['name'] = names
scr['mol'] = mols
scr['fp'] = fps

### 계산

In [225]:
sim_values = []
sim_names = []
sim_smiles = []

for fp in scr['fp']:
    sim_res = [DataStructs.FingerprintSimilarity(fp, fp2) for fp2 in ref['fp']]
    max_sim = max(sim_res)
    max_idx = sim_res.index(max_sim)
    max_name = ref['name'][max_idx]
    max_smiles = ref['smiles'][max_idx]
    
    max_sim = round(max_sim, 2)
    sim_values.append(max_sim)
    sim_names.append(max_name)
    sim_smiles.append(max_smiles)
    
df['fp_sim_max'] = sim_values
df['fp_sim_ref'] = sim_names
df['fp_sim_smiles'] = sim_smiles

In [226]:
filtered = df 
# filtered = df.loc[df['fp_sim_max'] >= 0.7]
filtered

,compound_id,smiles,Matched Ligand Sites,PhaseScreenScore,Query Title,Screen Title,r_phase_Shape_Sim,inchikey,chiral_removed_smiles,chiral_removed_inchikey,...,count_max_fused_rings,count_aromatic_rings,count_rings,count_contiguous_rotatable_bonds,hba+hbd,atom_N+O,magic_filter,fp_sim_max,fp_sim_ref,fp_sim_smiles
0,MCULE-6121281505,CC1=NC(N2CCOC(C3CC3)C2)=C(C#N)C(C)=C1C,NaN,NaN,"Compound 25 (S) (Page No: 24, Table No: 1)(que...",MCULE-6121281505,0.704059,RFRARHLSSWVGRY-UHFFFAOYSA-N,Cc1nc(N2CCOC(C3CC3)C2)c(C#N)c(C)c1C,RFRARHLSSWVGRY-UHFFFAOYSA-N,...,1,1,3,0,4,4,True,0.61,"Example 7 (Stereo) (Page No: 78, Table No: 1)",c1ccccc1N(C)C(=O)[C@@H]2C[C@H](O)CN2c3nc(C)cc(...
1,MCULE-5636307387,CC1=CC(C)=C(C#N)C(N2CCOC(C(=O)N3CCCCC3)C2)=N1,NaN,NaN,Example 1 (S) (Page No: 69)(query 13),MCULE-5636307387,0.702990,PTZZLQMSGISTDS-UHFFFAOYSA-N,Cc1cc(C)c(C#N)c(N2CCOC(C(=O)N3CCCCC3)C2)n1,PTZZLQMSGISTDS-UHFFFAOYSA-N,...,1,1,3,0,5,6,True,0.61,Example 42 (Stereo) (Page No: 96),Cc(c1)cccc1N(CC)C(=O)[C@@H]2C[C@H](N3CCOCC3)CN...
2,MCULE-9235853759,CCN(CC)C(=O)CSC1=NC(C2CC2)=CC(C(F)(F)F)=C1C#N,NaN,NaN,Example 1 (S) (Page No: 69)(query 13),MCULE-9235853759,0.682689,NGXPMIXCBUIZLY-UHFFFAOYSA-N,CCN(CC)C(=O)CSc1nc(C2CC2)cc(C(F)(F)F)c1C#N,NGXPMIXCBUIZLY-UHFFFAOYSA-N,...,1,1,2,1,4,4,True,0.46,"Example 152 (Stereo) (Page No: 174, Table No: 9)",Clc(c1)c(F)ccc1N(CC)C(=O)[C@@H]2C[C@H](N3CCOC[...
3,MCULE-5043437498,CC1=CC(OCC(=O)N2C3=CC=C(F)C=C3CCC2C)=CC(C)=C1Cl,A(2) A(-) H(3) H(5) H(4) R(9) R(10),1.760,"Compound 25 (S) (Page No: 24, Table No: 1)(que...",MCULE-5043437498,0.681150,UXCYHTXJTXSNFN-UHFFFAOYSA-N,Cc1cc(OCC(=O)N2c3ccc(F)cc3CCC2C)cc(C)c1Cl,UXCYHTXJTXSNFN-UHFFFAOYSA-N,...,2,2,3,1,2,3,True,0.51,"Compound 14 (Page No: 22, Table No: 1)",c1cc(F)ccc1N(C)C(=O)COc2cc(Cl)cc(c2)C(F)(F)F
4,MCULE-8476928603,CC1=CC(C)=C(C#N)C(OCC(=O)N2CCC3=CC(Br)=CC=C32)=N1,A(4) A(1) H(6) H(5) H(8) R(9) R(10),1.643,"Compound 25 (S) (Page No: 24, Table No: 1)(que...",MCULE-8476928603,0.673433,QYRSSNIHQJTDDR-UHFFFAOYSA-N,Cc1cc(C)c(C#N)c(OCC(=O)N2CCc3cc(Br)ccc32)n1,QYRSSNIHQJTDDR-UHFFFAOYSA-N,...,2,2,3,1,4,5,True,0.72,"Compound 35 (Page No: 26, Table No: 1)",c1cc(Br)ccc1N(C)C(=O)COc2c(C#N)c(C)c(Cl)c(n2)C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,MCULE-2533389215,CCC1=NC2=CC(C3CCCCN3C(=O)C3CC=CCC3)=NN2C(C(F)(...,NaN,NaN,"Compound 2 (Page No: 20, Table No: 1)(query 6)",MCULE-2533389215,0.600296,HTAFZCVTEOOLPE-UHFFFAOYSA-N,CCc1cc(C(F)(F)F)n2nc(C3CCCCN3C(=O)C3CC=CCC3)cc2n1,HTAFZCVTEOOLPE-UHFFFAOYSA-N,...,2,2,4,0,4,5,False,0.52,Example 69 (Stereo) (Page No: 120),Fc1c(Cl)c(F)ccc1N(C)C(=O)[C@H]2N(C(=O)[C@H]([C...
293,MCULE-9174449344,CC1=CC(C)=C(C#N)C(OC2=CC=CC=C2CN(C)C)=N1,NaN,NaN,Example 1 (S) (Page No: 69)(query 13),MCULE-9174449344,0.600112,YVCTWQUFQAEKCU-UHFFFAOYSA-N,Cc1cc(C)c(C#N)c(Oc2ccccc2CN(C)C)n1,YVCTWQUFQAEKCU-UHFFFAOYSA-N,...,1,2,2,0,4,4,True,0.45,"Compound 44 (Page No: 28, Table No: 1)",c1ccccc1N(C)C(=O)COc2c(C#N)c(C)c(Cl)c(n2)C
294,MCULE-3839498327,CC(SC1=NC(C2=CC=CC=C2)=CC(C(F)(F)F)=N1)C(=O)N1...,NaN,NaN,"Compound 2 (Page No: 20, Table No: 1)(query 6)",MCULE-3839498327,0.600111,SUURSINTCWOPFE-UHFFFAOYSA-N,CC(Sc1nc(-c2ccccc2)cc(C(F)(F)F)n1)C(=O)N1CCCc2...,SUURSINTCWOPFE-UHFFFAOYSA-N,...,2,3,4,1,4,4,False,0.49,"Example 150 (Stereo) (Page No: 173, Table No: 9)",Cc(c1)c(F)ccc1N(CC)C(=O)[C@@H]2C[C@H](N3CCOC(C...
295,MCULE-2677197970,CC1=CC(C(F)(F)F)=C(C#N)C(SCC(=O)NC2=CC=CC=C2C)=N1,A(3) A(1) H(7) H(5) H(6) R(9) R(10),1.532,Example 1 (S) (Page No: 69)(query 13),MCULE-2677197970,0.600041,YYZZGFFHLUDUST-UHFFFAOYSA-N,Cc1cc(C(F)(F)F)c(C#N)c(SCC(=O)Nc2ccccc2C)n1,YYZZGFFHLUDUST-UHFFFAOYSA-N,...,1,2,2,1,5,4,True,0.50,"Compound 49 (Page No: 29, Table No: 1)",N#Cc1cc(F)ccc1N(C)C(=O)CNc2c(C#N)c(C(F)(F)F)cc...


In [227]:
filtered.to_csv('out_fp_sim_filtered.csv', index=False)